In [1]:
import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'RPOSE'))
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/core'))
sys.path.append(os.path.join(os.getcwd() + '/segmentation'))

import pickle
import numpy as np 

# p = "/home/jonfrey/RPOSE/notebooks/Mode.MUTIPLE_INIT_POSES_data_final.pkl" # experiment with predicted flow and valid
p = "/home/jonfrey/RPOSE/notebooks/Mode.MUTIPLE_INIT_POSES_gt_valid_data_final.pkl"

# more noise
p = "/home/jonfrey/RPOSE/notebooks/Mode.MUTIPLE_INIT_POSES_gt_valid_more_noise_trained_data_final.pkl"

with open(p, 'rb') as f:
     data = pickle.load(f)
        
print( data.keys() )
for k in data.keys():
    try:
        data[k] = data[k][:,0]
    except:
        pass
import pandas as pd
df = pd.DataFrame.from_dict( data ) 
df

dict_keys(['add_s', 'adds', 'idx_arr', 'ratios_arr', 'valid_corrospondences', 'init_adds_arr', 'init_add_s_arr'])


,add_s,adds,idx_arr,ratios_arr,valid_corrospondences,init_adds_arr,init_add_s_arr
0,0.016085,0.010293,1.0,-4.146982,49050.0,0.005930,0.010871
1,0.042244,0.032048,17.0,-1.190897,25573.0,0.024075,0.033945
2,0.035910,0.024469,17.0,-0.798035,27579.0,0.003868,0.015589
3,0.004885,0.004885,18.0,-3.358136,31987.0,0.029031,0.029031
4,0.052646,0.018309,1.0,-2.567994,37278.0,0.023107,0.061368
...,...,...,...,...,...,...,...
14020,0.017245,0.017245,18.0,-11.531987,29115.0,0.046860,0.046860
14021,0.013108,0.007874,2.0,-1.101843,28689.0,0.029792,0.057417
14022,0.028598,0.011196,1.0,-3.109534,34853.0,0.020764,0.056532
14023,0.074144,0.008305,8.0,-0.696259,12087.0,0.033621,0.097913


In [2]:
with open('/home/jonfrey/PoseCNN-PyTorch/data_posecnn.pickle', 'rb') as handle:
    posecnn = pickle.load(handle)
data_posecnn = {}
data_posecnn['add_s'] = np.array( [d['distances_non'] for d in posecnn])
data_posecnn['adds'] = np.array( [d['distances_sys'] for d in posecnn])
data_posecnn['idx_arr'] = np.array( [d['cls_index']-1 for d in posecnn])
df_posecnn = pd.DataFrame.from_dict( data_posecnn ) 
df_posecnn 

,add_s,adds,idx_arr
0,0.010331,0.006101,0
1,0.015830,0.005267,5
2,0.082734,0.047821,13
3,0.127282,0.081377,18
4,0.058098,0.032544,19
...,...,...,...
14020,0.014305,0.007267,3
14021,0.021800,0.012786,5
14022,0.010457,0.004589,8
14023,0.050037,0.017271,14


In [3]:
from pose_estimation import compute_auc
sym = [20,19,18,15,12]

def auc( k,obj,df):
    return round( compute_auc( df[k][df['idx_arr']==obj].to_numpy() ),2)
def bold(a,b):
    if a > b:
        return f'\033[1m {a} \033[0m vs {b}'#  + str(a) +'\033[0m' + ' vs ' + str(b)
    return f' {a} vs \033[1m {b} \033[0m'


for obj in range(0,21):
    s = ""
    if obj in sym:
        s+= '\033[1m'
    s+= f'ADDS-AUC {obj}: '
    if obj in sym:
        s+= '\033[0m'

    a,b =  auc('adds',obj,df), auc('adds',obj,df_posecnn)
    s+= bold(a,b)
    if 'init_adds_arr' in df.keys():
        s+= f"   init: {auc('init_adds_arr',obj,df)}"
    print(s)
    
print("")
print("")
print("")


for obj in range(0,21):
    s = ""
    if obj in sym:
        s+= '\033[1m'
    s+= f'ADD-(S) AUC {obj}: '
    if obj in sym:
        s+= '\033[0m'
    
    a,b =  auc('add_s',obj,df), auc('add_s',obj,df_posecnn)
    s+= bold(a,b)
    if 'init_add_s_arr' in df.keys():
        s+= f"   init: {auc('init_add_s_arr',obj,df)}"
    print(s)

ADDS-AUC 0:  90.2  vs 87.84   init: 87.85
ADDS-AUC 1:  74.59 vs  80.48    init: 80.86
ADDS-AUC 2:  81.63  vs 74.1   init: 73.21
ADDS-AUC 3:  82.23 vs  83.94    init: 83.98
ADDS-AUC 4:  92.78  vs 90.09   init: 89.74
ADDS-AUC 5:  81.59 vs  87.52    init: 87.36
ADDS-AUC 6:  77.07  vs 70.02   init: 66.6
ADDS-AUC 7:  87.05 vs  88.59    init: 85.76
ADDS-AUC 8:  82.32 vs  85.97    init: 86.41
ADDS-AUC 9:  66.69 vs  87.0    init: 86.53
ADDS-AUC 10:  91.5  vs 86.21   init: 86.38
ADDS-AUC 11:  64.14 vs  65.36    init: 66.21
ADDS-AUC 12:  40.65 vs  73.06    init: 72.27
ADDS-AUC 13:  72.37 vs  76.28    init: 76.65
ADDS-AUC 14:  81.39  vs 74.83   init: 74.37
ADDS-AUC 15:  27.02  vs 25.29   init: 25.59
ADDS-AUC 16:  53.55 vs  62.96    init: 63.74
ADDS-AUC 17:  56.86 vs  71.95    init: 72.02
ADDS-AUC 18:  38.02 vs  41.26    init: 37.4
ADDS-AUC 19:  21.47 vs  30.48    init: 26.24
ADDS-AUC 20:  41.02 vs  85.09    init: 86.2



ADD-(S) AUC 0:  64.33  vs 61.56   init: 61.56
ADD-(S) AUC 1:  61.54 vs  61.9